In [1]:
!pip install stable-baselines3
!pip install pyglet
!pip install swig
!pip install box2d-py
!pip install importlib-metadata==4.13.0
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 180 kB 37.6 MB/s 
     |████████████████████████████████| 1.5 MB 53.2 MB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=b068a6e9f589a0ea557cfde06c645d5b35d3d45e476239f9105c1c6d578707ed
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 29.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.7 MB 33.3 MB/s 
Looking in indexes: https://pypi.org/simple, https:

In [2]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import DDPG
from stable_baselines3.common.monitor import Monitor
import os
import gym

from stable_baselines3.common.evaluation import evaluate_policy

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

from typing import Any, Dict
import torch
import torch.nn as nn

from stable_baselines3.common.callbacks import EvalCallback

import torch as th

In [3]:
# #########################
# # OPTIMISER HYPERPARAMS #
# #########################
# env_id = "LunarLanderContinuous-v2"
# #opti_env = gym.make(env_id)
# eval_envs = make_vec_env(env_id, n_envs=10)
# budget = 100_000

# log_dir = "data/save/"
# os.makedirs(log_dir, exist_ok=True)

# file_name = "ddpg_" + env_id
# log_file_name = log_dir + file_name

# N_TRIALS = 200  # Maximum number of trials
# N_STARTUP_TRIALS = 5  # Stop random sampling after N_STARTUP_TRIALS
# N_EVALUATIONS = 2  # Number of evaluations during training
# N_JOBS = 10 # Number of parallel jobs
# N_TIMESTEPS = int(1e5)  # Training budget
# EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
# N_EVAL_ENVS = 5
# N_EVAL_EPISODES = 10
# TIMEOUT = int(60 * 15)  # 15 minutes

# ENV_ID = "LunarLanderContinuous-v2"

# DEFAULT_HYPERPARAMS = {
#     "policy": "MlpPolicy",
#     "env": ENV_ID,
# }

# def sample_ddpg_params(trial: optuna.Trial) -> Dict[str, Any]:
#     """Sampler for DDPG hyperparameters."""
#     # Discount factor between 0.9 and 0.9999
#     gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
#     max_grad_norm = trial.suggest_float("max_grad_norm", 0.1, 8.0, log=True)
#     # 8, 16, 32, ... 1024
#     n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)

#     ### YOUR CODE HERE
#     # TODO:
#     # - define the learning rate search space [1e-5, 1] (log) -> `suggest_float`
#     # - define the network architecture search space ["tiny", "small"] -> `suggest_categorical`
#     # - define the activation function search space ["tanh", "relu"]
#     learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
#     net_arch = trial.suggest_categorical("archi", ["tiny", "small","medium","big","large","verybig"])
#     activation_fn = trial.suggest_categorical("activ fun", ["tanh", "relu"])

#     ### END OF YOUR CODE

#     # Display true values
#     trial.set_user_attr("gamma_", gamma)
#     trial.set_user_attr("n_steps", n_steps)

#     net_arch = {
#         "tiny": [32],
#         "small": [64, 64],
#         "medium": [256, 256],
#         "big": [400, 300],
#         "large": [256, 256, 256],
#         "verybig": [512, 512, 512],
#     }[net_arch]

#     activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

#     return {
#         # "n_steps": n_steps,
#         "gamma": gamma,
#         "learning_rate": learning_rate,
#         # "max_grad_norm": max_grad_norm,
#         "policy_kwargs": {
#             "net_arch": net_arch,
#             "activation_fn": activation_fn,
#         },
#     }

# class TrialEvalCallback(EvalCallback):
#     """Callback used for evaluating and reporting a trial."""

#     def __init__(
#         self,
#         eval_env: gym.Env,
#         trial: optuna.Trial,
#         n_eval_episodes: int = 10,
#         eval_freq: int = 10000,
#         deterministic: bool = True,
#         verbose: int = 0,
#     ):

#         super().__init__(
#             eval_env=eval_env,
#             n_eval_episodes=n_eval_episodes,
#             eval_freq=eval_freq,
#             deterministic=deterministic,
#             verbose=verbose,
#         )
#         self.trial = trial
#         self.eval_idx = 0
#         self.is_pruned = False

#     def _on_step(self) -> bool:
#         if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
#             super()._on_step()
#             self.eval_idx += 1
#             self.trial.report(self.last_mean_reward, self.eval_idx)
#             # Prune trial if need
#             if self.trial.should_prune():
#                 self.is_pruned = True
#                 return False
#         return True

# def objective(trial: optuna.Trial) -> float:

#     kwargs = DEFAULT_HYPERPARAMS.copy()

#     # Sample hyperparameters and update the keyword arguments
#     new_dict = sample_ddpg_params(trial)
#     kwargs.update(new_dict)

#     # Create the RL model
#     model = DDPG(**kwargs)
#     # Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`

#     # Create the callback that will periodically evaluate
#     # and report the performance using `N_EVAL_EPISODES` every `EVAL_FREQ`
#     eval_callback = TrialEvalCallback(eval_envs, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ)

#     ### END OF YOUR CODE

#     nan_encountered = False
#     try:
#         # Train the model
#         model.learn(N_TIMESTEPS, callback=eval_callback)
#     except AssertionError as e:
#         # Sometimes, random hyperparams can generate NaN
#         print(e)
#         nan_encountered = True
#     finally:
#         # Free memory
#         model.env.close()
#         eval_envs.close()

#     # Tell the optimizer that the trial failed
#     if nan_encountered:
#         return float("nan")

#     if eval_callback.is_pruned:
#         raise optuna.exceptions.TrialPruned()

#     return eval_callback.last_mean_reward

# # Set pytorch num threads to 1 for faster training
# th.set_num_threads(1)
# # Select the sampler, can be random, TPESampler, CMAES, ...
# sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# # Do not prune before 1/3 of the max budget is used
# pruner = MedianPruner(
#     n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
# )
# # Create the study and start the hyperparameter optimization
# study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

# try:
#     study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
# except KeyboardInterrupt:
#     pass
# except (AssertionError, ValueError) as e: 
#      # Sometimes, random hyperparams can generate NaN 
#      print(e)

# print("Number of finished trials: ", len(study.trials))

# print("Best trial:")
# trial = study.best_trial

# print(f"  Value: {trial.value}")

# print("  Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

# print("  User attrs:")
# for key, value in trial.user_attrs.items():
#     print(f"    {key}: {value}")

# # Write report
# study.trials_dataframe().to_csv("study_results_ddpg_cartpole.csv")

In [4]:
# env = gym.make("LunarLanderContinuous-v2")
env = make_vec_env("LunarLanderContinuous-v2", n_envs=1)
#newEnv = Monitor(env, "logs/log.txt")

model = DDPG("MlpPolicy", env, verbose=1,  gamma=0.99, 
                                          learning_rate=0.001, 
                                          policy_kwargs={"net_arch":[256, 256], 
                                                         "activation_fn":nn.ReLU}
                                          )

model.learn(total_timesteps=400_000)

Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 106      |
|    ep_rew_mean     | -635     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 62       |
|    time_elapsed    | 6        |
|    total_timesteps | 426      |
| train/             |          |
|    actor_loss      | 9.44     |
|    critic_loss     | 26.6     |
|    learning_rate   | 0.001    |
|    n_updates       | 308      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 109      |
|    ep_rew_mean     | -608     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 68       |
|    time_elapsed    | 12       |
|    total_timesteps | 869      |
| train/             |          |
|    actor_loss      | 13.3     |
|    critic_loss     | 27.5     |
|    learning_rate   | 0.001    |
|    n_updates       | 818    

In [22]:
model.learn(total_timesteps=50_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 153      |
|    ep_rew_mean     | 148      |
| time/              |          |
|    episodes        | 4        |
|    fps             | 161      |
|    time_elapsed    | 3        |
|    total_timesteps | 612      |
| train/             |          |
|    actor_loss      | -107     |
|    critic_loss     | 13.2     |
|    learning_rate   | 0.001    |
|    n_updates       | 450685   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 164      |
|    ep_rew_mean     | 211      |
| time/              |          |
|    episodes        | 8        |
|    fps             | 139      |
|    time_elapsed    | 9        |
|    total_timesteps | 1312     |
| train/             |          |
|    actor_loss      | -107     |
|    critic_loss     | 15       |
|    learning_rate   | 0.001    |
|    n_updates       | 451387   |
--------------

In [23]:
testing = gym.make("LunarLanderContinuous-v2")
tst = evaluate_policy(model, testing, return_episode_rewards=True)

In [24]:
from numpy import mean, std
rewards = tst[0]
print( "min:",min(rewards) , "max:",max(rewards), "mean:", mean(rewards), "std:", std(rewards) )

min: 231.0236644141165 max: 299.0195253687804 mean: 263.94839236893944 std: 21.73415186710514


In [25]:
#min: 22.985810140147805 max: 264.1808621772134 mean: 219.35430867912436 std: 67.34621227805947
#min: 140.176276633807 max: 315.7162433904367 mean: 262.61416412449296 std: 46.35215411253695
#min: 231.0236644141165 max: 299.0195253687804 mean: 263.94839236893944 std: 21.73415186710514

In [26]:
# Viz
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.11).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [27]:
# # Set up fake display; otherwise rendering will fail
# import os
# os.system("Xvfb :1 -screen 0 1024x768x24 &")
# os.environ['DISPLAY'] = ':1'

# import base64
# from pathlib import Path

# from IPython import display as ipythondisplay

# def show_videos(video_path='', prefix=''):
#   """
#   Taken from https://github.com/eleurent/highway-env

#   :param video_path: (str) Path to the folder containing videos
#   :param prefix: (str) Filter the video, showing only the only starting with this prefix
#   """
#   html = []
#   for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
#       video_b64 = base64.b64encode(mp4.read_bytes())
#       html.append('''<video alt="{}" autoplay 
#                     loop controls style="height: 400px;">
#                     <source src="data:video/mp4;base64,{}" type="video/mp4" />
#                 </video>'''.format(mp4, video_b64.decode('ascii')))
#   ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

# from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

# def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
#   """
#   :param env_id: (str)
#   :param model: (RL model)
#   :param video_length: (int)
#   :param prefix: (str)
#   :param video_folder: (str)
#   """
#   eval_env = DummyVecEnv([lambda: gym.make(env_id)])
#   # Start the video at step=0 and record 500 steps
#   eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
#                               record_video_trigger=lambda step: step == 0, video_length=video_length,
#                               name_prefix=prefix)

#   obs = eval_env.reset()
#   for _ in range(video_length):
#     action, _ = model.predict(obs, deterministic=True)
#     obs, _, _, _ = eval_env.step(action)

#   # Close the video recorder
#   eval_env.close()

In [28]:
# record_video('LunarLanderContinuous-v2', model, video_length=500, prefix='ddpg-moon')

In [29]:
# show_videos('videos', prefix='ppo-moon')

In [31]:
model.save("ddpg_luna2r")